# Open AIs Agent Framework

https://openai.com/index/new-tools-for-building-agents/

## Sessions

In [1]:
from agents import set_trace_processors
set_trace_processors([]) # disable trace processors for this notebook

In [2]:

import os
from agents import Agent, OpenAIChatCompletionsModel, Runner, SQLiteSession
from openai import AsyncOpenAI, AsyncAzureOpenAI

dotenv.load_dotenv()

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]
deployment = os.environ["AZURE_OPENAI_DEPLOYMENT"]

client = AsyncAzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-08-01-preview")

model = OpenAIChatCompletionsModel(
    model=deployment,
    openai_client=client
)


session_id = "conversation_123"
session = SQLiteSession(session_id)

haiku_agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    model=model
)

result = await Runner.run(haiku_agent, "Tell me about recursion in programming.", session=session)
print(result.final_output)



Function calls itself,  
Dividing problems in parts,  
Stack keeps track of time.


In [3]:
result = await Runner.run(haiku_agent, "Translate this into german.", session=session)
print(result.final_output)


Funktion ruft sich selbst,  
Teilt Probleme in Teile,  
Der Stack zählt die Zeit.


## Tool Calls

In [4]:
from pydantic import BaseModel

from agents import Agent, Runner, function_tool


class Weather(BaseModel):
    city: str
    temperature_range: str
    conditions: str


@function_tool
def get_weather(city: str) -> Weather:
    '''Get the current weather for a given city'''

    print(f"[debug] get_weather called for city: {city}")
    return Weather(city=city, temperature_range="14-20C", conditions="Sunny with wind.")

@function_tool
def schnupsi(a: int, b: int) -> int:
    '''Add two numbers
    Args:
        a: first number
        b: second number
    Returns:        sum of a and b
    '''
    print(f"[debug] schnupsi called with a: {a}, b: {b}")
    return a + b


haiku_agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    model=model,
    tools=[get_weather,schnupsi]

)

result = await Runner.run(haiku_agent, input="What's the weather in Tokyo?")
print(result.final_output)


[debug] get_weather called for city: Tokyo
Tokyo's skies are clear,  
Temperatures mild and bright,  
Winds dance in the sun.  


In [5]:
result = await Runner.run(haiku_agent, input="Was ist 5 * 10?")
print(result.final_output)

[debug] schnupsi called with a: 5, b: 10
Fünf mal zehn, ja,  
Die Antwort ist fünfzehn,  
Mathematik spricht.


# How can we integrate an MCP Server?

In [ ]:
from agents.mcp import MCPServer, MCPServerStreamableHttp
from agents.model_settings import ModelSettings


mcp_server = MCPServerStreamableHttp(
    name="Streamable HTTP Python Server",
    params={
        "url": "http://localhost:3333/mcp",
    })

await mcp_server.connect()

print(await mcp_server.list_tools())   # List available tools on the MCP server

haiku_agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model=model,
        mcp_servers=[mcp_server],
        model_settings=ModelSettings(tool_choice="required"),
    )

# Use the `add` tool to add two numbers
result = await Runner.run(haiku_agent, input="Add these numbers: 7 and 22.")

print(result.final_output)



[Tool(name='add', title=None, description='Add two numbers', inputSchema={'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}, outputSchema={'properties': {'result': {'type': 'integer'}}, 'required': ['result'], 'type': 'object', 'x-fastmcp-wrap-result': True}, icons=None, annotations=None, meta={'_fastmcp': {'tags': []}}), Tool(name='fetch_url', title=None, description='Fetch / download the content of a given URL. Return the HTML content as a string.', inputSchema={'properties': {'url': {'type': 'string'}}, 'required': ['url'], 'type': 'object'}, outputSchema={'properties': {'result': {'type': 'string'}}, 'required': ['result'], 'type': 'object', 'x-fastmcp-wrap-result': True}, icons=None, annotations=None, meta={'_fastmcp': {'tags': []}})]
Seven plus twenty,  
The answer is twenty-nine,  
Math brings joy to life.  
